In [1]:
import sys
sys.path.append('/work/utilis')

from lib_to_use import *
from general_functions import *
from paths import *

/root/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Torch version:  2.6.0+cu124
Pyterrier version:  0.13.0
/work/utilis/lib_to_use.py:32: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [2]:
qrels_train = load_qrels(qrels_train_path)
qrels_val = load_qrels(qrels_val_path)
qrels_test = load_qrels(qrels_test_path)
qrels_train_df = qrels_to_df(qrels_train)
qrels_val_df = qrels_to_df(qrels_val)
qrels_test_df = qrels_to_df(qrels_test)

In [3]:
with open(test_processed_path, "rb") as f:
    df_test = pickle.load(f)

with open(val_processed_path, "rb") as f:
    df_val = pickle.load(f)

with open(train_processed_path, "rb") as f:
    df_train = pickle.load(f)

with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/df_all_processed.pkl", "rb") as f:
    df_all = pickle.load(f)
df_all

,qid,query,title,timestamp,score,views,favorite,user_id,user_questions,user_answers,tags,rel_ids,rel_scores,rel_timestamps,best_answer,text,docno
0,academia_100305,cnr research unit staf centr nation de la rech...,What are CNRS research units and how are they ...,1513009820,14,2484,2.0,1106095,"[workplace_40845, workplace_40899, workplace_9...","[travel_45926, travel_46391, travel_47403, tra...","[funding, france]",[academia_100217],[1],"[1512814966, 1513014615, 1513020822]",academia_100217,wikipedia friend read cnr wikipag look cnr off...,academia_100217
1,academia_100456,free freedom altern publon review journal allo...,Is there a free (as in freedom) alternative to...,1513191752,13,1117,2.0,1106095,"[workplace_40845, workplace_40899, workplace_9...","[travel_45926, travel_46391, travel_47403, tra...","[peer-review, open-access]",[academia_100462],[1],"[1513205016, 1536615064, 1553005541, 1615097827]",academia_100462,look question coupl time sinc publon becam act...,academia_100462
2,academia_103390,search stackexchang citat googl scholar possib...,Search for StackExchange citations with Google...,1517935259,2,157,1.0,1532620,"[writers_27613, writers_29562, sound_42166, so...","[skeptics_39944, philosophy_3098, philosophy_9...","[citations, google-scholar]",[academia_103391],[1],[1517936080],academia_103391,accord answer public cite stack exchangeleg an...,academia_103391
3,academia_10481,reproduc research corpor ident univers drive u...,Reproducible research and corporate identity,1370509865,18,372,1.0,1106095,"[academia_1698, academia_1772, academia_1911, ...","[academia_1699, academia_1700, academia_1701, ...","[copyright, creative-commons]",[academia_10499],[1],"[1370596608, 1370601095]",academia_10499,univers enforc corpor ident sinc 1990 describ ...,academia_10499
4,academia_10649,advantag second mark uk portion case work seco...,Advantages of second marking,1371471877,6,1235,2.0,1106095,"[academia_1698, academia_1772, academia_1911, ...","[academia_1699, academia_1700, academia_1701, ...",[assessment],[academia_10650],[1],"[1371477146, 1371477156, 1371552185]",academia_10650,doubl mark mani role mostli ensur accuraci fai...,academia_10650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10126,politics_62030,main criterion use european commiss comput cov...,What are the main criteria used by the Europea...,1611233512,6,260,NaN,3305982,"[workplace_82935, workplace_103798, workplace_...","[workplace_106814, workplace_110533, vegetaria...","[covid-19, european-commission]",[politics_62578],[1],[1613416543],politics_62578,written question mep european commiss topic oc...,politics_62578
10127,politics_74901,happen five star movement m5 deputi sinc 2018 ...,What happened to most of Five Star Movement (M...,1661026202,5,134,NaN,4403071,"[skeptics_39707, skeptics_39869, skeptics_3987...","[workplace_41095, workplace_41175, travel_1373...","[parliament, parties, italy]",[politics_74939],[1],[1661161283],politics_74939,parliamentif want visual chart chang look work...,politics_74939
10128,scifi_251849,dumbledor know ron saw mirror eris philosoph s...,How did Dumbledore know what Ron saw in the Mi...,1627048749,6,404,NaN,1077413,"[writers_6201, writers_6562, writers_24286, wo...","[writers_48048, scifi_7887, scifi_8424, scifi_...",[harry-potter],[scifi_251851],[1],[1627050021],scifi_251851,dumbledor room ron describ saw mirror erisedwh...,scifi_251851
10129,gaming_386587,resid facad talk languag understood first get ...,Do the residents of Facade talk in a language ...,1622833418,2,140,NaN,923667,"[sports_14511, sports_15814, sports_15832, spo...","[pets_16769, gaming_38304, gaming_38330, gamin...",[nier-replicant-1.22474487139],[gaming_386590],[1],[1622834001],gaming_386590,accord wiki languag facad resid origin languag...,gaming_386590


In [4]:
# Create a PyTerrier index
index_path = "./pyterrier_index"
# indexer = pt.IterDictIndexer(index_path, overwrite=True,meta={'docno': 50, 'text': 10000})

# # Index only the document collection (answers)
# corpus = df_all[['docno', 'text']].to_dict('records')
# # rimuovi i duplicati docno
# index_ref = indexer.index(corpus)

# Load the index previously built (using the IndexFactory)
index_ref = pt.IndexFactory.of(index_path)

terrier-assemblies 5.11 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done
Java started (triggered by IndexFactory.of) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


In [5]:
from sentence_transformers import SentenceTransformer, util

# 1. Initialize the models
minilm_model_name = "sentence-transformers/all-MiniLM-L6-v2"
distilbert_qa_model_name = "sentence-transformers/msmarco-distilbert-base-tas-b"  # Ottimizzato per Q&A
minilm_model = SentenceTransformer(minilm_model_name)
distilbert_qa_model = SentenceTransformer(distilbert_qa_model_name)

# Precompute a dictionary mapping docno to text to avoid repeated DataFrame filtering.
doc_text_dict = df_all.set_index('docno')['text'].to_dict()

class Reranker(pt.Transformer):
    def __init__(self, model, num_results):
        self.model = model
        self.num_results = num_results
        
    def transform(self, topics_and_res: pd.DataFrame) -> pd.DataFrame:
        """
        Re-rank retrieval results using a neural model (e.g., MiniLM or DistilBERT QA).
        This version optimizes CPU and RAM usage by:
         - Using a precomputed dictionary to map docno -> text,
         - Encoding document texts in batch.
         
        Expects topics_and_res to have at least the columns:
            'qid', 'query', 'docno'
        (Il campo 'text' verrà recuperato dal dizionario precomputato)
        """
        # Copy the input DataFrame
        results_df = topics_and_res.copy()
        
        # Assicurati che il DataFrame contenga 'qid', 'query' e 'docno'
        for col in ['qid', 'query', 'docno']:
            if col not in results_df.columns:
                raise ValueError(f"Missing required column: {col}")
        
        # Process each group (per query) in batch
        for qid, group in results_df.groupby('qid'):
            # Get the query text (assumed identical for the same qid)
            query_text = str(group['query'].iloc[0])
            
            # Retrieve document texts using the precomputed dictionary.
            # Se un documento non viene trovato, viene usata una stringa vuota.
            docs = [doc_text_dict.get(docno, "") for docno in group['docno']]
            
            # Batch encode the query (single string) and the list of document texts.
            query_embedding = self.model.encode(query_text, convert_to_tensor=True, normalize_embeddings=True)
            doc_embeddings = self.model.encode(docs, convert_to_tensor=True, normalize_embeddings=True)
            
            # Compute cosine similarities in batch (questo è molto efficiente)
            similarities = util.pytorch_cos_sim(query_embedding, doc_embeddings)[0]
            
            # Update the 'score' column with the new MiniLM similarity scores
            group_indices = group.index
            results_df.loc[group_indices, 'score'] = similarities.cpu().numpy()
            
            # Compute rank: ordina i documenti in ordine decrescente del punteggio
            results_df.loc[group_indices, 'rank'] = (-results_df.loc[group_indices, 'score']).argsort() + 1
        
        # Sort overall results by 'qid' e 'rank'
        results_df = results_df.sort_values(['qid', 'rank'])
        
        # Optionally, mantieni solo i top k risultati per query se specificato
        if self.num_results:
            results_df = results_df.groupby('qid').head(self.num_results).reset_index(drop=True)
        
        return results_df

In [6]:
# 3. Create the pipeline
# bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25", num_results=100, metadata=['docno', 'text'])  # Retrieve more documents for reranking
bm25 = pt.terrier.Retriever(index_ref, wmodel="BM25", num_results=100, metadata=['docno', 'text'], controls={"k_1": 1, "b": 1})  # Retrieve more documents for reranking
minilm_reranker = Reranker(minilm_model, num_results=100)
distilbert_qa_reranker = Reranker(distilbert_qa_model, num_results=50)

# Combine them into a pipeline
pipeline_minilm = bm25 >> pt.apply.query_level(minilm_reranker)
pipeline_distilbert_qa = bm25 >> pt.apply.query_level(distilbert_qa_reranker)

In [7]:
# Prepara le query nel formato corretto
queries_df = pd.DataFrame(df_val[['qid', 'query']])
queries_df['qid'] = queries_df['qid'].astype(str)

# Se vuoi anche i risultati di solo BM25 per confronto
bm25_results = bm25.transform(queries_df)
bm25_results.head()

,qid,docid,docno,text,rank,score,query
0,academia_143743,66,academia_28111,mean fellowship phdstudent extern award ok lis...,0,24.197326,answer question one ask histor research came a...
1,academia_143743,14,academia_12035,short answer free authorship also known gift a...,1,23.709126,answer question one ask histor research came a...
2,academia_143743,147,academia_76526,conceptu pointsfor two year ive work draft pap...,2,21.791199,answer question one ask histor research came a...
3,academia_143743,3441,law_5712,first law order taken accur depict new york tr...,3,21.178890,answer question one ask histor research came a...
4,academia_143743,6525,scifi_117801,question specif ask canon scifi fantasi im qui...,4,18.967244,answer question one ask histor research came a...


In [8]:
# Run the pipeline
# results_distilbert_qa = pipeline_distilbert_qa.transform(df_val[['qid', 'query']])

# results_distilbert_qa = distilbert_qa_reranker.transform(bm25_results)
# print("Re-ranked results:")
# print(results_distilbert_qa.head())

In [9]:
# with open ("/work/PIR_data_unzip/PIR_data/answer_retrieval/saved_results/bm25_distilbert_qa_results.pkl", "wb") as f:
#     pickle.dump(results_distilbert_qa, f)

In [10]:
# results_minilm = pipeline_minilm.transform(queries_df)
results_minilm  = minilm_reranker.transform(bm25_results)
# with open ("/work/PIR_data_unzip/PIR_data/answer_retrieval/saved_results/bm25_minilm_results.pkl", "wb") as f:
#     pickle.dump(results_minilm, f)
print("Re-ranked results:")
print(results_minilm.head())

Re-ranked results:
               qid  docid             docno  \
0  academia_143743   6227  workplace_114128   
1  academia_143743     10    academia_11592   
2  academia_143743     16   academia_122342   
3  academia_143743   6864    politics_41447   
4  academia_143743    145    academia_75098   

                                                text  rank     score  \
0  dont think inappropri would take care word que...     1  0.493768   
1  there certain amount judgment need exercis one...     2  0.556254   
2  let make two point first order write research ...     3  0.570407   
3  base announc latest version relev legisl altho...     4  0.392424   
4  ive lot undergrad lab physic unlik corvu id sa...     5  0.417857   

                                               query  
0  answer question one ask histor research came a...  
1  answer question one ask histor research came a...  
2  answer question one ask histor research came a...  
3  answer question one ask histor research ca

In [11]:
# Per valutare i risultati BM25+distilbert-qa
eval_distil = pt.Evaluate(results_distilbert_qa, qrels_val_df, metrics=["P_1", "recall_100", "map_cut_100", "ndcg_cut_3"])
print("BM25+distilbert-qa Evaluation:", eval_distil)

NameError: name 'results_distilbert_qa' is not defined

In [13]:
# Per valutare i risultati BM25
eval_bm25 = pt.Evaluate(bm25_results, qrels_val_df, metrics=["P_1", "recall_100", "map_cut_100", "ndcg_cut_3"])
print("BM25 Evaluation:", eval_bm25)

# # Per valutare i risultati BM25+MiniLM
eval_minilm = pt.Evaluate(results_minilm, qrels_val_df, metrics=["P_1", "recall_100", "map_cut_100", "ndcg_cut_3"])
print("BM25+MiniLM Evaluation:", eval_minilm)

BM25 Evaluation: {'P_1': 0.7142857142857143, 'recall_100': 0.9285714285714286, 'map_cut_100': 0.7731338936073157, 'ndcg_cut_3': 0.7722282426749296}


NameError: name 'results_minilm' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=144ec01f-394f-474f-b507-b786ab13b472' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>